In [56]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn import preprocessing, metrics
from sklearn.feature_extraction.text import TfidfVectorizer
np.random.seed(1)

In [15]:
test_df = pd.read_csv("test.csv")

In [16]:
test_df

,article_number,article_words,topic
0,9501,"world,complet,pharmaceut,tianjin,tianjin,chin,...",IRRELEVANT
1,9502,"copy,sunday,weekend,ec,friday,eu,includ,limit,...",IRRELEVANT
2,9503,"heavy,heavy,gabriel,morn,morn,equit,cent,cent,...",FOREX MARKETS
3,9504,"research,jess,hit,anticip,comput,comput,comput...",IRRELEVANT
4,9505,"provid,provid,luxembourg,court,court,case,opin...",IRRELEVANT
...,...,...,...
495,9996,"symantec,soon,interfac,provid,provid,comput,co...",IRRELEVANT
496,9997,"hit,hit,hit,hit,hit,hit,hit,hit,hit,beat,beat,...",SPORTS
497,9998,"cent,cent,cent,cent,match,declin,act,rate,rate...",MONEY MARKETS
498,9999,"cnmv,cnmv,stock,count,count,week,group,friday,...",SHARE LISTINGS


In [17]:
train_df = pd.read_csv("training.csv")

In [18]:
train_df

,article_number,article_words,topic
0,1,"open,absent,cent,cent,cent,stock,inflow,rate,k...",FOREX MARKETS
1,2,"morn,stead,end,end,day,day,day,patch,patch,pat...",MONEY MARKETS
2,3,"socc,socc,world,world,recent,law,fifa,fifa,fif...",SPORTS
3,4,"open,forint,forint,forint,forint,cent,cent,ste...",FOREX MARKETS
4,5,"morn,complet,weekend,minut,minut,minut,arrow,d...",IRRELEVANT
...,...,...,...
9495,9496,"cloud,provid,hope,centur,erupt,rule,recent,sou...",DEFENCE
9496,9497,"stock,stock,stock,declin,access,week,worry,blo...",IRRELEVANT
9497,9498,"rate,million,million,belarus,dollar,dollar,nov...",FOREX MARKETS
9498,9499,"flow,bullet,bullet,bullet,bullet,bullet,bullet...",IRRELEVANT


# Preprocessing
*base on the work of Raymond*

Since `imblearn` is not a module installed on cse machine, did not apply.


In [31]:
train_x = train_df.article_words
train_y = train_df.topic
test_x = test_df.article_words
test_y = test_df.topic

en_train_y = preprocessing.LabelEncoder().fit_transform(train_y)
en_test_y = preprocessing.LabelEncoder().fit_transform(test_y)
categories=list(set(train_df['topic']))
print(categories)

['IRRELEVANT', 'BIOGRAPHIES PERSONALITIES PEOPLE', 'SCIENCE AND TECHNOLOGY', 'DEFENCE', 'SHARE LISTINGS', 'SPORTS', 'FOREX MARKETS', 'ARTS CULTURE ENTERTAINMENT', 'DOMESTIC MARKETS', 'MONEY MARKETS', 'HEALTH']


In [27]:
vectorizer = TfidfVectorizer().fit(train_x)
train_vectors = vectorizer.transform(train_x)
test_vectors = vectorizer.transform(test_x)
print(train_vectors.shape,test_vectors.shape)
dim = train_vectors.shape[1]

(9500, 35822) (500, 35822)


# Training Model

In [64]:
def network():
    model = Sequential()
    model.add(Dense(11, input_dim=35822, activation='relu'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [65]:
estimator = KerasClassifier(build_fn=network, epochs=100, batch_size=5, verbose=0)

In [66]:
estimator.fit(train_vectors, train_y)

In [67]:
test_pred = estimator.predict(test_vectors)
metrics.confusion_matrix(test_y, test_pred)

array([[  3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 15,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 13,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 48,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 14,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [266,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 69,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  7,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 60,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])